In [35]:
from src import Modeler, Processor, Reader
import pandas as pd
from datetime import datetime
from sklearn import metrics, model_selection, preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
import time

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns 
sns.set()

In [37]:
# Using the US Census regions from here: https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf
south_atlantic_states = [
    10   # Delaware
    , 11 # DC
    , 12 # Florida
    , 13 # Georgia
    , 24 # Maryland
    , 37 # North Carolina
    , 45 # South Carolina
    , 51 # Virginia
    , 54 # West Virginia
]
state_filter = south_atlantic_states

In [39]:
state_dict = {
    'code' : ['10', '11', '12', '13', '24', '37', '45', '51', '54']
    , 'state_name' : ['Delware', 'DC', 'Florida', 'Georgia', 'Maryland', 'North Carolina', 'South Carolina', 'Virginia', 'West Virginia']
}

state_code_map = pd.DataFrame(data=state_dict)

## Loading data

In [41]:
# Instantiate preprocessor
preprocessor = Processor.PreProcessor(state_filter=state_filter)

In [43]:
# Put data into a df to work with
df = preprocessor.get_processed_data()

/Users/elizabethmartens/cs5644/project/projectGitRepo/src/Processor.py:37: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  merged_df = self.init_cdc_data().merge(self.init_vaccinations_data(), how='left', on=['date', 'FIPS'])
/Users/elizabethmartens/cs5644/project/projectGitRepo/src/Processor.py:37: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  merged_df = self.init_cdc_data().merge(self.init_vaccinations_data(), how='left', on=['date', 'FIPS'])


In [ ]:
# Export data to csv so that the data does not have to be recreated each run
df.to_csv('COVID_Analysis_Data.csv',index=False)
# Show data
df.head()

In [ ]:
# Load data
#df = pd.read_csv("COVID_Analysis_Data.csv", dtypes={'date':''})

In [ ]:
df.shape

## Split into train and test

In [ ]:
modeler = Modeler.Modeler()

In [ ]:
# Split data into training and test dataset:
train_end_date = '2021-06-14'
X_train, y_train_cases, y_train_deaths, X_test, y_test_cases, y_test_deaths = modeler.split_train_test(df, train_end_date)

### Unpivoted data

In [ ]:
unpivoted_df = preprocessor.get_processed_data_without_fips_as_columns(set_current_data=False)

In [ ]:
unpivoted_cases = unpivoted_df[['date', 'FIPS', 'cases']].copy()
unpivoted_cases['STATE'] = unpivoted_cases['FIPS'].apply(lambda x: x[:2])
unpivoted_cases = unpivoted_cases.merge(state_code_map, left_on='STATE', right_on='code', how='left')

In [ ]:
unpivoted_cases_test = unpivoted_cases[unpivoted_cases['date'] > train_end_date]

In [ ]:
unpivoted_deaths = unpivoted_df[['date', 'FIPS', 'deaths']].copy()
unpivoted_deaths['STATE'] = unpivoted_deaths['FIPS'].apply(lambda x: x[:2])
unpivoted_deaths = unpivoted_deaths.merge(state_code_map, left_on='STATE', right_on='code', how='left')

In [ ]:
unpivoted_deaths_test = unpivoted_deaths[unpivoted_deaths['date'] > train_end_date]

## Plotting functions

In [ ]:
def plot_predictions(y_test, y_pred, model_name=None):
    plt.figure(figsize=(9,6))
    plt.scatter(y_test, y_pred, color='none', edgecolor='b')
    xymin = min(np.min(y_test), np.min(y_pred))
    xymax = max(np.max(y_test), np.max(y_pred))
    plt.plot([xymin, xymax],[xymin, xymax], color="r", linestyle="--")
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    if model_name:
        plt.title(f'{model_name}: Actuals vs Predicted', fontsize=16)
    else:
        plt.title('Actuals vs Predicted', fontsize=16)
    plt.show()
    
def plot_predictions_w_state(y_test, y_pred, unpivoted_test_df, model_name=None):
    plt.figure(figsize=(9,6))
    sns.scatterplot(x=y_test, y=y_pred, hue=unpivoted_test_df['state_name'])
    xymin = min(np.min(y_test), np.min(y_pred))
    xymax = max(np.max(y_test), np.max(y_pred))
    plt.plot([xymin, xymax],[xymin, xymax], color="r", linestyle="--")
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    if model_name:
        plt.title(f'{model_name}: Actuals vs Predicted by state', fontsize=16)
    else:
        plt.title('Actuals vs Predicted by state', fontsize=16)
    plt.show()
    
def plot_residuals(X_test, residuals, model_name=None):
    plt.figure(figsize=(18, 6))
    sns.scatterplot(x=df['date'], y=residuals, color='none', edgecolor='r')
    if model_name:
        plt.title(f'{model_name}: Residuals', fontsize=16)
    else:
        plt.title('Residuals', fontsize=16)
    plt.show()
    
def plot_residuals_w_state(X_test, residuals, unpivoted_test_df, model_name=None):
    plt.figure(figsize=(18, 6))
    sns.scatterplot(x=df['date'], y=residuals, hue=unpivoted_test_df['state_name'])
    if model_name:
        plt.title(f'{model_name}: Residuals by state', fontsize=16)
    else:
        plt.title('Residuals by state', fontsize=16)
    plt.show()

## Modeling functions

In [ ]:
def run_full_pipeline(model, X_train, y_train, X_test, y_test, unpivoted_test_df, model_name=None):
    if model_name:
        print(f'{model_name}')
    # Cross validate and report scores:
    scores = modeler.cv_model(model, X_train, y_train)
    print('Cross-Validation Scores:')
    for k, v in scores.items():
        print(f'\t{k}: {round(v[0],2)} +/- {round(v[1],2)}')
    # Test data
    start = time.time()
    y_pred, rmse, r2, fit_time, fit_predict_time = modeler.test_model(model, X_train, y_train, X_test, y_test)
    residuals = (y_test - y_pred)
    print(f'Time to train model: {fit_time}')
    # Plots
    plot_predictions(y_test, y_pred, model_name)
    plot_residuals(X_test, residuals, model_name)
    plot_predictions_w_state(y_test, y_pred, unpivoted_test_df, model_name)
    plot_residuals_w_state(X_test, residuals, unpivoted_test_df, model_name)
    return y_pred, residuals, rmse, r2, fit_time

# Gradient Boosting

In [ ]:
gb_model = GradientBoostingRegressor()

In [ ]:
# Cases
gb_model.fit(X_train, y_train_cases)

y_pred_cases = gb_model.predict(X_test)

In [ ]:
mse = metrics.mean_squared_error(y_test_cases, y_pred_cases)
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

In [ ]:
# Adapted from https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#plot-training-deviance
number_of_estimators = 100
test_score = np.zeros((number_of_estimators,), dtype=np.float64)
for i, y_pred in enumerate(gb_model.staged_predict(X_test)):
    test_score[i] = gb_model.loss_(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title("Deviance")
plt.plot(
    np.arange(number_of_estimators) + 1,
    gb_model.train_score_,
    "b-",
    label="Training Set Deviance",
)
plt.plot(
    np.arange(number_of_estimators) + 1, test_score, "r-", label="Test Set Deviance"
)
plt.legend(loc="upper right")
plt.xlabel("Boosting Iterations")
plt.ylabel("Deviance")
fig.tight_layout()
plt.show()

